<a href="https://colab.research.google.com/github/DJlee331/Julia_Practice_DJ/blob/main/BVAR_JUlia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# using Pkg
# Pkg.add("FredData")
# Pkg.add("Dates")
# Pkg.add("Distributions")

In [2]:
using DataFrames, CSV
using Statistics, LinearAlgebra
using Random, Distributions
Random.seed!(4412)

TaskLocalRNG()

In [3]:
using FredData
using Dates

In [4]:
fred = FredData.Fred("adeee66f40c0687c4decaba2442fcaff") # Replace YOUR_FRED_API_KEY with your key

FRED API Connection
	url: https://api.stlouisfed.org/fred/
	key: adeee66f40c0687c4decaba2442fcaff


In [5]:
# 2. 시리즈 ID로 데이터 요청 (예: GDP)
DF = get_data(fred, "PCEPI").data[:,[:date,:value]]; # PCE
rename!(DF,:value => :pce);
series = get_data(fred, "PCEPILFE").data[:,[:date,:value]]; # PCE core
rename!(series,:value => :pcecore);
DF = outerjoin(DF,series, on = :date);
series = get_data(fred, "CPIAUCSL").data[:,[:date,:value]]; # CPI
rename!(series,:value => :cpi);
DF = outerjoin(DF,series, on = :date);
series = get_data(fred, "CPILFESL").data[:,[:date,:value]]; # CPI Core
rename!(series,:value => :cpicore);
DF = outerjoin(DF,series, on = :date);
#series = get_data(fred, "T10YIE").data[:,[:date,:value]]; # 10year BEI
#rename!(series,:value => :bei);
#DF = outerjoin(DF,series, on = :date);
series = get_data(fred, "INDPRO").data[:,[:date,:value]]; # Industrial Production
rename!(series,:value => :indpro);
DF = outerjoin(DF,series, on = :date);
series = get_data(fred, "UNRATE").data[:,[:date,:value]]; # Unemployment Rate
rename!(series,:value => :unemp);
DF = outerjoin(DF,series, on = :date);
series = get_data(fred, "PAYEMS").data[:,[:date,:value]]; # Total NonFarm employees
rename!(series,:value => :employ);
DF = outerjoin(DF,series, on = :date);
series = get_data(fred, "DFF").data[:,[:date,:value]]; # Federal funds effective rate
rename!(series,:value => :ffr);
DF = outerjoin(DF,series, on = :date);
series = get_data(fred, "DGS10").data[:,[:date,:value]]; # Market yield on Us Treasury 10y
rename!(series,:value => :year10);
DF = outerjoin(DF,series, on = :date);
series = get_data(fred, "DAAA").data[:,[:date,:value]]; # Moody's Seasoned Aaa Corporate bond yield
rename!(series,:value => :corpbond);
DF = outerjoin(DF,series, on = :date);
#series = get_data(fred, "MSPUS").data[:,[:date,:value]]; # Median Sales Price of Houses Sold for the United States
#rename!(series,:value => :hpi);
#DF = outerjoin(DF,series, on = :date);
series = get_data(fred, "TOTALSA").data[:,[:date,:value]]; # Total Vehicle Sales
rename!(series,:value => :vehicle);
DF = outerjoin(DF,series, on = :date);
series = get_data(fred, "WTISPLC").data[:,[:date,:value]]; # Spot Crude Oil Price: West Texas Intermediate (WTI)
rename!(series,:value => :oil);
DF = outerjoin(DF,series, on = :date);

┌ Warning: Metadata 'notes' not returned from server.
└ @ FredData /Users/dj/.julia/packages/FredData/5M7x4/src/get_data.jl:77


In [6]:
sort!(DF, :date);
DF.month = Date.(year.(DF.date),month.(DF.date),1);
df_m = combine(groupby(DF, :month),
    names(DF, Not([:date, :month])) .=> (x -> mean(filter(!isnan, skipmissing(x)))) .=> names(DF, Not([:date,:month]))
);
rename!(df_m,:month =>:date);
start_date = Date("1984-01-01")
end_date   = Date("2019-12-01")

DF_F = df_m[(df_m.date .>= start_date) .& (df_m.date .<= end_date), :];
DF_F[[1:5; end-4:end],:]

Row,date,pce,pcecore,cpi,cpicore,indpro,unemp,employ,ffr,year10,corpbond,vehicle,oil
,Date,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1984-01-01,47.564,47.121,102.1,102.5,53.0088,8.0,92673.0,9.55839,11.6743,12.2,14.21,29.69
2,1984-02-01,47.884,47.418,102.6,102.8,53.2486,7.8,93157.0,9.58759,11.8421,12.0835,14.381,30.145
3,1984-03-01,48.041,47.598,102.9,103.2,53.5028,7.8,93429.0,9.91323,12.3191,12.5736,14.21,30.761
4,1984-04-01,48.224,47.822,103.3,103.7,53.8233,7.7,93792.0,10.291,12.6335,12.8085,14.357,30.618
5,1984-05-01,48.285,47.933,103.5,104.1,54.1104,7.4,94098.0,10.3194,13.4086,13.2827,14.856,30.52
6,2019-08-01,103.67,103.797,256.036,263.877,102.781,3.6,151171.0,2.12581,1.62636,2.97727,17.882,54.81
7,2019-09-01,103.733,103.873,256.43,264.388,102.46,3.5,151365.0,2.043,1.6995,3.032,17.72,56.95
8,2019-10-01,103.917,104.028,257.155,264.97,101.588,3.6,151460.0,1.82968,1.70682,3.00818,17.049,53.96
9,2019-11-01,104.024,104.092,257.879,265.548,102.149,3.6,151668.0,1.55333,1.81211,3.05737,17.549,57.03


In [7]:
log_level = vec([1 2 3 4 5 7  12]);
# names(DF_F,Not(:date))[log_level]
transform!(DF_F, names(DF_F,Not(:date))[log_level] .=> ByRow(x -> log(x) * 100) .=> names(DF_F,Not(:date))[log_level] );
DF_F[[1:5; end-4:end],:]

Row,date,pce,pcecore,cpi,cpicore,indpro,unemp,employ,ffr,year10,corpbond,vehicle,oil
,Date,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1984-01-01,386.208,385.272,462.595,462.986,397.046,8.0,1143.68,9.55839,11.6743,12.2,14.21,339.081
2,1984-02-01,386.878,385.9,463.084,463.279,397.497,7.8,1144.2,9.58759,11.8421,12.0835,14.381,340.602
3,1984-03-01,387.205,386.279,463.376,463.667,397.973,7.8,1144.5,9.91323,12.3191,12.5736,14.21,342.625
4,1984-04-01,387.586,386.749,463.764,464.15,398.571,7.7,1144.88,10.291,12.6335,12.8085,14.357,342.159
5,1984-05-01,387.712,386.98,463.957,464.535,399.103,7.4,1145.21,10.3194,13.4086,13.2827,14.856,341.838
6,2019-08-01,464.121,464.244,554.532,557.548,463.26,3.6,1192.62,2.12581,1.62636,2.97727,17.882,400.387
7,2019-09-01,464.182,464.317,554.686,557.742,462.947,3.5,1192.74,2.043,1.6995,3.032,17.72,404.217
8,2019-10-01,464.359,464.466,554.968,557.962,462.092,3.6,1192.81,1.82968,1.70682,3.00818,17.049,398.824
9,2019-11-01,464.462,464.528,555.249,558.18,462.644,3.6,1192.94,1.55333,1.81211,3.05737,17.549,404.358


In [8]:
DF_Q =copy(DF_F);
DF_Q.quarter = Date.(year.(DF_F.date),quarterofyear.(Date.(DF_F.date)).*3 ,1); # 분기 말월 1일 기준
DF_Q = combine(groupby(DF_Q, :quarter),names(DF_Q, Not([:date, :quarter])) .=> mean)
rename!(DF_Q, :quarter => :date);
DF_Q[[1:4;end-3:end],:]

Row,date,pce_mean,pcecore_mean,cpi_mean,cpicore_mean,indpro_mean,unemp_mean,employ_mean,ffr_mean,year10_mean,corpbond_mean,vehicle_mean,oil_mean
,Date,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1984-03-01,386.764,385.817,463.018,463.311,397.505,7.86667,1144.13,9.6864,11.9452,12.2857,14.267,340.769
2,1984-06-01,387.729,386.982,463.957,464.535,399.047,7.43333,1145.24,10.5573,13.2017,13.2134,14.6287,341.336
3,1984-09-01,388.497,387.916,464.823,465.776,399.73,7.43333,1146.22,11.3913,12.867,12.9896,14.332,337.085
4,1984-12-01,389.112,388.625,465.681,466.783,399.778,7.3,1147.11,9.26425,11.7442,12.3506,14.6727,331.025
5,2019-03-01,463.328,463.438,553.491,556.518,463.479,3.86667,1191.93,2.40172,2.65369,3.83095,17.258,400.29
6,2019-06-01,463.886,463.859,554.223,556.956,462.9,3.63333,1192.29,2.39743,2.33379,3.59662,17.572,408.864
7,2019-09-01,464.126,464.23,554.553,557.537,462.904,3.6,1192.61,2.19057,1.79498,3.10082,17.7613,403.174
8,2019-12-01,464.518,464.581,555.252,558.166,462.392,3.6,1192.93,1.64466,1.79393,3.02423,17.363,404.139


Estimate a Bayesian VAR

In [9]:
# When all Latent variables were observable
Y = Matrix(DF_F[:,Not(([:date]))]);
p=6; # Number of lags
T,k = size(Y)

(432, 12)

In [10]:
function prior_init(Y::Matrix{Float64},p::Int,log_level::Vector{Int},lambda1::Float64=0.09, lambda2::Float64=4.3,lambda3::Int=1,lambda4::Float64=2.7,lambda5::Float64=4.3,includeconstant::Bool=true)
# Initialize prior(Minnesota prior, initial values were optimal values from Song&Schorfheide 2015) 
  T,k = size(Y);
  # prior for Innovation Covariance matrix Σ
  S0 = zeros(k,k);
  # Estimate standard errors of AR(1) for each variable
  for i=1:k
    y = Y[:,i];
    epsilon = y[2:end]-hcat(ones(T-1,1), y[1:end-1])*(hcat(ones(T-1,1), y[1:end-1])\y[2:end]);
    S0[i,i] = epsilon'epsilon/T;
  end

  # prior for coefficients Aⱼs
  B0 = zeros(k,k*p);
  for xx=1:k
    if any(xx .== log_level)
      B0[xx,xx] = 1; # For nonstationary variables
    else
      B0[xx,xx] = 0.5; # For stationary variables
    end
  end
  if includeconstant;
    B0 = hcat(B0,zeros(k,1)); # Add constant term
  end
  
  Omega = ones(1,k*p+1);
  Omega[end] =1e+6;
  Omega[1:end-1] = lambda1^2 *(kron([j^-2lambda2 for j=1:p], diag(S0)));
  Omega = Diagonal(Omega[1:end]);
  V0 = kron(S0,Omega)
  return B0, V0, S0, Omega, MN_hyper(lambda1,lambda2,lambda3,lambda4,lambda5)
end

struct MN_hyper # Store Hyperparameters of Minnesota prior(notations based on Song&Schorfheide 2015)
  lambda1::Float64       # overall tightness
  lambda2::Float64       # the rate at which the prior variance on higher-order lag coefficients decays
  lambda3::Int    # Dispersion of the prior on the innovation covariance matrix
  lambda4::Float64    # Sum of coefficient prior
  lambda5::Float64    # the extent to which co-persistence restrictions are imposed on the VAR coefficients
  
end

In [11]:
B0, V0, S0, Omega, hyperparameters = prior_init(Y, p, log_level, 0.09, 4.3, 1, 2.7, 4.3, true);

In [12]:
function gibbs_bvar(Y::Matrix, p::Int, B0,V0,S0, hyperparameters::MN_hyper, n_draws::Int = 2000, burnin::Int = 1000)
    T, k = size(Y)
    T0 = 2*p; # pre-sample size
    Tnobs = T - T0;
    Y_dummy, X_dummy = make_dummy_observations(hyperparameters::MN_hyper,  Y::Matrix{Float64}, p::Int, T0::Int)
    X, Y_target = make_lag_matrix(Y, p)
    # premom = vcat(mean(Y[1:p,:],dims=1),std(Y[1:p,:],dims=1));
    # # Y_dummy, X_dummy = make_dummy_observations(k,p,hyperparameters,premom,1)
    # make_dummy_observations(hyperparameters,  Y, p)
    Td = size(Y_dummy, 1);
    X_f = vcat(X_dummy,X[T0+1-p:end,:]);
    Y_f = vcat(Y_dummy, Y_target[T0+1-p:end,:]);

    V0_inv = inv(V0)
    #B_samples = zeros(burnin+n_draws, k, k*p+1)  # n_draws × k × (k*p+1)
    #Σ_samples = zeros(burnin+n_draws, k, k)  # n_draws × k × k
    B_samples = Array{Float64}(undef, n_draws-burnin, k, k*p+1)  # 3D array
    Σ_samples = Array{Float64}(undef, n_draws-burnin, k, k)  # 3D array

    # 초기값
    B_draw = reshape(vec(X_f\Y_f) , k*p+1,k)'
    B_draw_init = transpose(X_f\Y_f);
    XtX_inv = cholesky(X_f'*X_f) \ I;  # safer than inv()
    U_X = cholesky(Symmetric(XtX_inv)).U     
for s in 1:n_draws
    # Posterior of Σ | B
    E = Y_f - X_f * B_draw';
    Σ_post = Wishart(T+Td-k-1-(T0-p),E'E); # Posterior of Σ | Y;
    Σ_draw = inv(rand(Σ_post));
    # posterior of B | Σ 
    # B_V =  Matrix(Symmetric(kron(Σ_draw,XtX_inv)));
    # B_draw = reshape(vec(X_f\Y_f) + cholesky(B_V).U *randn(prod(size(B0))), k,k*p+1);        
    B_draw = B_draw_init + cholesky(Matrix(Symmetric(Σ_draw))).L *randn((size(B0))) *U_X; # A Secret Sauce for Super-fast computation, by ChatGPT
    # Save results
#    B_samples[s,:,:] = reshape(B_draw',1,k,k*p+1);
    #Σ_samples[s,:,:] = reshape(Σ_draw,1,k,k);
    if s>burnin
    @views B_samples[s-burnin, :, :] .= B_draw;
    @views Σ_samples[s-burnin, :, :] .= Σ_draw;
    end
 if mod(s,5000) == 0
        println("Iteration: ", s)
    end
  
end
    return B_samples, Σ_samples;
end

function make_lag_matrix(Y::Matrix, p::Int)
    T, n = size(Y)
    X = hcat([Y[p - i + 1:end - i, :] for i in 1:p]...)  # concaternating lag matrices
    X = hcat(X, ones(T - p))  # 상수항
    Y_target = Y[p+1:end, :]
    return X, Y_target
end
#FYI...
#@time transpose(X_f\Y_f) + cholesky(Matrix(Symmetric(Σ_draw))).L *randn((size(B0))) *cholesky(Matrix(Symmetric((XtX_inv)))).U; # 4.8ms / draw
#@time reshape(vec(X_f\Y_f) + cholesky(B_V).U *randn(prod(size(B0))), k,k*p+1);        # 12.8ms/draw 
# @time rand(MvNormal(vec(X_f\Y_f), B_V))  # 13.0 ms/draw

function make_dummy_observations(hyperparameters::MN_hyper,  Y::Matrix{Float64}, p::Int, T0::Int)
    λ1, λ2, λ3, λ4, λ5 = hyperparameters.lambda1, hyperparameters.lambda2, hyperparameters.lambda3, hyperparameters.lambda4, hyperparameters.lambda5;
    T,k = size(Y)
    T0 = 2*p; # pre-sample size
    x_ = mean(Y[1:T0,:],dims=1); # pre-sample mean
    s_ = std(Y[1:T0,:],dims=1); # pre-sample standard deviation

    dsize = 1 + k*(p+λ3+1);
    breakss = Vector{Int}(undef,5); # zeros(5,1);
    ydu = zeros(dsize,k);
    xdu = zeros(dsize,k*p+1);
    # Dummies for the coefficients of the first lag
    sig = s_.*Matrix(I,k,k);;
    ydu[1:k,:] = λ1 * sig;
    xdu[1:k,:] = hcat( λ1*sig, zeros(k,(p-1)*k+1) );
    breakss[1] = k;
    # Dummies for the coefficients of the remaining lags
    if p>1;
       ydu[k+1:k*p,:] = zeros((p-1)*k,k);
       j = 1;
       while j <=p-1
        xdu[k+1+(j-1)*k:k+j*k,:] = [zeros(k,j*k) λ1*sig*((j+1)^λ2) zeros(k,(p-1-j)*k+1)];
        j = j+1;
       end
       breakss[2] = k + (p-1)*k;
    else
        breakss[2] = breakss[1];
    end
   # Dummies for the covariance matrix of error terms
    ydu[breakss[2]+1:breakss[2]+λ3*k,:] = kron(ones(λ3,1),sig);
   breakss[3] = breakss[2]+λ3*k;

    # dummies for the coefficents of the constant term
    lammean = λ4*x_;
    ydu[breakss[3]+1,:] = lammean;
    xdu[breakss[3]+1,:] = [kron(ones(1,p),lammean) λ4];
    breakss[4] = breakss[3]+1;

    # dummies for the covariance matrix of coefficients of different lags
     mumean = λ5* x_.*Matrix(I,k,k);
     ydu[breakss[4]+1:breakss[4]+k,:] = Matrix(mumean);
     xdu[breakss[4]+1:breakss[4]+k,:] = [kron(ones(1,p),mumean) zeros(k,1)];
     breakss[5] = breakss[4]+k;

    return ydu, xdu;
end


make_dummy_observations (generic function with 1 method)

In [13]:
@time B_post, sig_post = gibbs_bvar(Y, p, B0,V0,S0, hyperparameters, 55000, 5000);

Iteration: 5000
Iteration: 10000
Iteration: 15000
Iteration: 20000
Iteration: 25000
Iteration: 30000
Iteration: 35000
Iteration: 40000
Iteration: 45000
Iteration: 50000
Iteration: 55000
 12.675197 seconds (10.53 M allocations: 8.425 GiB, 9.31% gc time, 32.01% compilation time)


In [14]:
# Construct a Pseudo-Mixed frequency Dataset (Act like we can only observe quarterly PCE)
Pseudo_DF = sort(outerjoin(DF_F[:,names(DF_F,Not([:pce,:pcecore]))],DF_Q[:,[:date,:pce_mean,:pcecore_mean]], on = :date),:date);
select!(Pseudo_DF, [:date, :pce_mean, :pcecore_mean],:);
Pseudo_DF[[1:6;end-5:end],:]

Row,date,pce_mean,pcecore_mean,cpi,cpicore,indpro,unemp,employ,ffr,year10,corpbond,vehicle,oil
,Date,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,1984-01-01,missing,missing,462.595,462.986,397.046,8.0,1143.68,9.55839,11.6743,12.2,14.21,339.081
2,1984-02-01,missing,missing,463.084,463.279,397.497,7.8,1144.2,9.58759,11.8421,12.0835,14.381,340.602
3,1984-03-01,386.764,385.817,463.376,463.667,397.973,7.8,1144.5,9.91323,12.3191,12.5736,14.21,342.625
4,1984-04-01,missing,missing,463.764,464.15,398.571,7.7,1144.88,10.291,12.6335,12.8085,14.357,342.159
5,1984-05-01,missing,missing,463.957,464.535,399.103,7.4,1145.21,10.3194,13.4086,13.2827,14.856,341.838
6,1984-06-01,387.729,386.982,464.15,464.919,399.466,7.2,1145.61,11.0617,13.5629,13.549,14.673,340.01
7,2019-07-01,missing,missing,554.44,557.322,462.503,3.7,1192.46,2.4029,2.05909,3.29318,17.682,404.917
8,2019-08-01,missing,missing,554.532,557.548,463.26,3.6,1192.62,2.12581,1.62636,2.97727,17.882,400.387
9,2019-09-01,464.126,464.23,554.686,557.742,462.947,3.5,1192.74,2.043,1.6995,3.032,17.72,404.217


In [ ]:
# Index for quarterly and monthly variables
quarterly_names = names(Pseudo_DF,Not(:date))[[any(ismissing, Pseudo_DF[!, col]) for col in names(Pseudo_DF,Not(:date))]];
monthly_names = names(Pseudo_DF,Not(:date))[[~any(ismissing, Pseudo_DF[!, col]) for col in names(Pseudo_DF,Not(:date))]];
isquarterly = [any(ismissing, Pseudo_DF[!, i]) for i in names(Pseudo_DF, Not(:date))];

In [ ]:
#Pkg.add("Interpolations")
using Interpolations

In [ ]:
# 각 월의 일 수 차이로 변환 후 내삽
Y = Matrix(DF_Q[:,2:end]);
x_monthly = Dates.value.(Pseudo_DF.date .- Pseudo_DF.date[1]);
x_quarterly = Dates.value.(DF_Q.date .- DF_Q.date[1]);
results = map(i -> extrapolate(interpolate((x_quarterly,), Y[:, i], Gridded(Linear())), Line()).(x_monthly), findall(isquarterly));
results = hcat(results...);
#Pseudo_DF[!,[:pce_interpolated, :pcecore_interpolated]] = results;
Pseudo_DF.pce_interpolated = results[:,1];
Pseudo_DF.pcecore_interpolated = results[:,1];


In [ ]:
Nq = sum(isquarterly); # Number of quarterly variables
Nm = sum(.~isquarterly); # Number of monthly variables

In [ ]:
# Matrices for collecting draws from Posterior Density
nsim = 1000;
T,k = size(Z_init_DF)
T0 = Int(2*p); 
Tnobs = T - T0;
Sigmap    = zeros(nsim,k,k);
Phip      = zeros(nsim,k*p+1,k);
Cons      = zeros(nsim,k);
lstate    = zeros(nsim,Nq,Tnobs);
YYactsim  = zeros(nsim,4,k);
XXactsim  = zeros(nsim,4,k*p+1);
likesim   = zeros(nsim,1);
zmddsim   = zeros(nsim,1);    
priorsim  = zeros(nsim,2);
parasim   = zeros(nsim,Int((k*p+1)*k + k*(k+1)/2));
At_mat    = zeros(Tnobs,Nq*(p+1));
Pt_mat    = zeros(Tnobs,(Nq*(p+1))^2);
Atildemat = zeros(nsim,Nq*(p+1));
Ptildemat = zeros(nsim,Nq*(p+1),Nq*(p+1));
loglh   = 0;
counter = 0;

In [ ]:
Z_init_DF = select(Pseudo_DF[1:end-1,names(Pseudo_DF,Not([:pce_mean,:pcecore_mean]))], [:date],:);
# Quarterly data는 제일 뒤에 오너라
#YDATA = Matrix(Z_init_DF[:,2:end]);
YQ = Matrix{Float64}(Z_init_DF[T0+1:end,[:pce_interpolated,:pcecore_interpolated]]);
YM = Matrix{Float64}(Z_init_DF[T0+1:end,monthly_names]);
Z_init_DF[[1:6;end-4:end],:]

In [ ]:
YDATA = hcat(YM,YQ)
B_post_init, sig_post_init = gibbs_bvar(YDATA, p, B0,V0,S0, hyperparameters, 5000, 1000);

In [ ]:
    Phi = copy(Matrix(B0'));
    sigma = copy(S0);
    Phi = reshape(median(B_post_init,dims=1),k,k*p+1)';
    sigma = reshape(median(sig_post_init,dims=1),k,k);
    phi_qm = zeros(Nm * p, Nq);
    for i in 1:p
        phi_qm[(Nm*(i-1)+1):(Nm*i), :] = Phi[(i-1)*(Nm+Nq)+1:(i-1)*(Nm+Nq)+Nm, Nm+1:end];
    end

    phi_qq = zeros(Nq * p, Nq);
    for i in 1:p
        phi_qq[(Nq*(i-1)+1):(Nq*i), :] = Phi[(i-1)*(Nm+Nq)+Nm+1:i*(Nm+Nq), Nm+1:end];
    end

    phi_qc = Phi[end, Nm+1:end];

    phi_mm = zeros(Nm * p, Nm);
    for i in 1:p
        phi_mm[(Nm*(i-1)+1):(Nm*i), :] = Phi[(i-1)*(Nm+Nq)+1:(i-1)*(Nm+Nq)+Nm, 1:Nm];
    end

    phi_mq = zeros(Nq * p, Nm);
    for i in 1:p
        phi_mq[(Nq*(i-1)+1):(Nq*i), :] = Phi[(i-1)*(Nm+Nq)+Nm+1:i*(Nm+Nq), 1:Nm];
    end

    phi_mc = Phi[end, 1:Nm];

    sig_mm = sigma[1:Nm, 1:Nm];
    sig_mq = 0.5 * (sigma[1:Nm, Nm+1:end] + sigma[Nm+1:end, 1:Nm]')
    sig_qm = 0.5 * (sigma[Nm+1:end, 1:Nm] + sigma[1:Nm, Nm+1:end]');
    sig_qq = sigma[Nm+1:end, Nm+1:end];

    # Transition Equation Matrices
    GAMMAs = vcat(hcat(phi_qq', zeros(Nq, Nq)), hcat(Matrix(I, p*Nq, p*Nq), zeros(p*Nq, Nq)));
    GAMMAz = vcat(phi_qm', zeros(p*Nq, p*Nm));
    GAMMAc = vcat(phi_qc, zeros(p*Nq,1));
    GAMMAu = vcat(Matrix(I, Nq, Nq), zeros(p*Nq, Nq));

    # Measurement Equation Matrices
    LAMBDAs = vcat(hcat(zeros(Nm, Nq), phi_mq'), (1/3) * hcat(Matrix(I, Nq, Nq), Matrix(I, Nq, Nq), Matrix(I, Nq, Nq), zeros(Nq, Nq*(p-2))));
    LAMBDAz = vcat(phi_mm', zeros(Nq, p*Nm));
    LAMBDAc = vcat(phi_mc, zeros(Nq));
    LAMBDAu = vcat(Matrix(I, Nm, Nm), zeros(Nq, Nm));

    # W matrix and transformed LAMBDA terms
    Wmatrix = hcat(Matrix(I, Nm, Nm), zeros(Nm, Nq));
    LAMBDAs_t = Wmatrix * LAMBDAs;
    LAMBDAz_t = Wmatrix * LAMBDAz;
    LAMBDAc_t = Wmatrix * LAMBDAc;
    LAMBDAu_t = Wmatrix * LAMBDAu;

In [ ]:
init_var = zeros(Nq * (p + 1), Nq * (p + 1));
for _ in 1:(p + 1)
    init_var = GAMMAs * init_var * GAMMAs' + GAMMAu * sig_qq * GAMMAu';
end
# YM = Matrix(Z_init_DF[:, names(Z_init_DF,Not([:date, :pce_interpolated, :pcecore_interpolated]))]);
# YQ = Matrix(Z_init_DF[:, [:pce_interpolated, :pcecore_interpolated]]);
Zm_init = zeros(T0 - p - 1, Nm * p);
for i in 1:p
    Zm_init[:, (i - 1) * Nm + 1 : i * Nm] = YM[(p + 1 - (i - 1)) : (T0 - i), :];
end
init_mean = vec(YQ[1:p+1,:]');

In [ ]:
function vm_initialize(GAMMAs, GAMMAz, GAMMAc, GAMMAu, LAMBDAs, LAMBDAz, LAMBDAc, LAMBDAu, LAMBDAs_t, LAMBDAz_t, LAMBDAc_t, LAMBDAu_t,
    sig_qq, sig_mm, sig_qm, sig_mq, Zm, YDATA, init_mean, init_var, p::Int,T0::Int, nv::Int, Nm::Int)
    # Unpack specification
    nex_ =1;

    # Initialization
    At = init_mean
    Pt = init_var

    for t in (p + 2):T0
        At1 = At
        Pt1 = Pt

        alphahat = GAMMAs * At1 + GAMMAz * Zm[t - p - 1, :] + GAMMAc ;
        Phat = Matrix(Symmetric(GAMMAs * Pt1 * GAMMAs' + GAMMAu * sig_qq * GAMMAu')) ;

        if iszero(mod(t, 3))
            # Quarterly observation
            yhat = LAMBDAs * alphahat + LAMBDAz * Zm[t - p - 1, :] + LAMBDAc
            nut = YDATA[t, :] - yhat

            Ft = LAMBDAs * Phat * LAMBDAs' + LAMBDAu * sig_mm * LAMBDAu' +
                 LAMBDAs * GAMMAu * sig_qm * LAMBDAu' +
                 LAMBDAu * sig_mq * GAMMAu' * LAMBDAs'
            Ft = Matrix(Symmetric(Ft));

            Xit = LAMBDAs * Phat + LAMBDAu * sig_mq * GAMMAu'
        else
            # Monthly observation
            yhat = LAMBDAs_t * alphahat + LAMBDAz_t * Zm[t - p - 1, :] + LAMBDAc_t
            nut = YDATA[t, 1:Nm] - yhat

            Ft = LAMBDAs_t * Phat * LAMBDAs_t' + LAMBDAu_t * sig_mm * LAMBDAu_t' +
                 LAMBDAs_t * GAMMAu * sig_qm * LAMBDAu_t' +
                 LAMBDAu_t * sig_mq * GAMMAu' * LAMBDAs_t'
            Ft = 0.5 * (Ft + Ft')

            Xit = LAMBDAs_t * Phat + LAMBDAu_t * sig_mq * GAMMAu'
        end

        At = alphahat + Xit' * inv(Ft) * nut
        Pt = Phat - Xit' * inv(Ft) * Xit
    end

    return At, Pt
end

In [ ]:
init_At, init_Pt = vm_initialize(GAMMAs, GAMMAz, GAMMAc, GAMMAu, LAMBDAs, LAMBDAz, LAMBDAc, LAMBDAu, LAMBDAs_t, LAMBDAz_t, LAMBDAc_t, LAMBDAu_t,
    sig_qq, sig_mm, sig_qm, sig_mq, Zm_init  , YDATA, init_mean, init_var, p::Int,T0::Int, k::Int, Nm::Int);

Kalman Filter

In [ ]:
# Lagged Monthly Observations
T,k = size(Z_init_DF[:,2:end]);
nobs = T-T0;
Zm   = zeros(nobs,Nm*p);

In [ ]:
nv=k;
nsim=1000;
kn = nv * (p + 1)
counter = 0
AT_draws = []
Atildemat = zeros(nsim, kn)
Ptildemat = zeros(nsim, kn, kn)
Tnew = 2;


For unbalanced dataset

In [ ]:
T0+nobs

In [ ]:

At = init_At
Pt = init_Pt
loglh = 0.0

At_mat = zeros(nobs, length(init_At))
Pt_mat = zeros(nobs, length(init_At)^2)

for t in 1:nobs    
    if iszero(mod(t + T0, 3))
        At1 = At
        Pt1 = Pt;
        #Forecasting
        alphahat = GAMMAs * At1 + GAMMAz * Zm[t, :] + GAMMAc
        Phat = Matrix(Symmetric( GAMMAs * Pt1 * GAMMAs' + GAMMAu * sig_qq * GAMMAu'));
        yhat = LAMBDAs * alphahat + LAMBDAz * Zm[t, :] + LAMBDAc
        nut = vcat(YM[t, :], YQ[t, :]) - yhat
        Ft = Matrix(Symmetric(LAMBDAs * Phat * LAMBDAs' + LAMBDAu * sig_mm * LAMBDAu' +
                    LAMBDAs * GAMMAu * sig_qm * LAMBDAu' + LAMBDAu * sig_mq * GAMMAu' * LAMBDAs'));
        loglh -= 0.5 * length(nut) * log(2π) + 0.5 * logdet(Ft) + 0.5 *only( nut' * inv(Ft) * nut)
        Xit = LAMBDAs * Phat + LAMBDAu * sig_mq * GAMMAu'
    else
        alphahat = GAMMAs * At1 + GAMMAz * Zm[t,:] + GAMMAc;
       Phat     = Matrix(Symmetric(GAMMAs * Pt1 * GAMMAs' + GAMMAu * sig_qq * GAMMAu'));
        yhat = LAMBDAs_t * alphahat + LAMBDAz_t * Zm[t, :] + LAMBDAc_t
        nut = YM[t, :] - yhat
        Ft = Matrix(Symmetric(LAMBDAs_t * Phat * LAMBDAs_t' + LAMBDAu_t * sig_mm * LAMBDAu_t' +
                     LAMBDAs_t * GAMMAu * sig_qm * LAMBDAu_t' + LAMBDAu_t * sig_mq * GAMMAu' * LAMBDAs_t'));
        loglh -= (0.5 * length(nut) * log(2π) + 0.5 * logdet(Ft) + 0.5 * only(nut' * inv(Ft) * nut))
        Xit = LAMBDAs_t * Phat + LAMBDAu_t * sig_mq * GAMMAu'
    end
    
    At = alphahat + Xit' * inv(Ft) * nut
    Pt = Phat - Xit' * inv(Ft) * Xit
    
    At_mat[t, :] = At'
    Pt_mat[t, :] = reshape(Pt, 1, :)
end

#=
Atildemat[j,:] = At_mat[nobs,:];
Pt_last  = reshape(Pt_mat[nobs,:],Nq*(p+1),Nq*(p+1));
Ptildemat[j,:,:] = Pt_last;
=#

# Measurement equation Z-matrix
j=1; # Simulation 마다 업데이트됨
Z1 = hcat(Matrix(I, Nm, Nm), zeros(Nm, kn - Nm))
Z2 = zeros(Nq, kn)
for bb in 1:Nq
    for ll in 1:3
        Z2[bb, ll * Nm + (ll - 1) * Nq + bb] = 1/3;
    end
end
ZZ = vcat(Z1, Z2)

# Build initial state BAt from posterior
BAt = Matrix{Float64}(undef, 0, 1)
for rr in 1:(p + 1)
    state_part = Atildemat[j, (rr - 1) * Nq + 1:rr * Nq]
    BAt = vcat(BAt, vcat(YM[end - rr + 1, :], state_part))
end

# Build initial covariance BPt
BPt = zeros(kn, kn)
for rr in 1:(p + 1)
    for vv in 1:(p + 1)
        r1 = rr * Nm + (rr - 1) * Nq + 1
        r2 = rr * (Nm + Nq)
        v1 = vv * Nm + (vv - 1) * Nq + 1
        v2 = vv * (Nm + Nq)
        BPt[r1:r2, v1:v2] .= Ptildemat[j, (rr - 1) * Nq + 1:rr * Nq, (vv - 1) * Nq + 1:vv * Nq]
    end
end

BAt_mat=zeros(T,kn);
BPt_mat=zeros(T,kn^2);

BAt_mat[nobs,:] = BAt;
BPt_mat[nobs,:] = reshape(BPt,1,kn^2);

# Define Companion Form
PHIF = zeros(kn, kn)
IF = Matrix(I, nv, nv)
for i in 1:p
    PHIF[i * nv + 1:(i + 1) * nv, (i - 1) * nv + 1:i * nv] = IF
end
PHIF[1:nv, 1:nv * p] = Phi[1:end-1, :]'

# Constant and Covariance term
CONF = vcat(Phi[end, :], zeros(nv * p))
SIGF = zeros(kn, kn)
SIGF[1:nv, 1:nv] = sigma

loglh = 0.0
# Filter Loop
for t in nobs + 1:Tnobs
    kkk = t - nobs
    ND = delif(YDATA[nobs + T0 + kkk, :]', index_NY[:, kkk])
    NZ = delif(ZZ, index_NY[:, kkk])

    BAt1 = BAt
    BPt1 = BPt

    # Forecast step
    Balphahat = PHIF * BAt1 + CONF
    BPhat = PHIF * BPt1 * PHIF' + SIGF
    BPhat = 0.5 * (BPhat + BPhat')

    Byhat = NZ * Balphahat
    Bnut = ND - Byhat
    BFt = NZ * BPhat * NZ'
    BFt = 0.5 * (BFt + BFt')

    loglh -= 0.5 * length(ND) * log(2π) + 0.5 * logdet(BFt) + 0.5 * Bnut' * inv(BFt) * Bnut

    # Update step
    BAt = Balphahat + BPhat * NZ' * inv(BFt) * Bnut
    BPt = BPhat - BPhat * NZ' * inv(BFt) * (BPhat * NZ')'

    BAt_mat[t, :] .= BAt[:]
    BPt_mat[t, :] .= reshape(BPt, 1, kn^2)
end

AT_draw = zeros(Tnew+1,kn);
u,s,v = svd(reshape(BPt_mat[Tnobs,:],kn,kn));
Pchol = u*Diagonal(s.^0.5)
AT_draw[end,:] = BAt_mat[Tnobs,:]+ (Pchol*randn(kn,1));

 # === Backward Kalman Smoother === #
 for i in 1:Tnew
    BAtt = BAt_mat[Tnobs - i, :]'
    BPtt = reshape(BPt_mat[Tnobs - i, :], kn, kn)

    BPhat = PHIF * BPtt * PHIF' + SIGF
    BPhat = 0.5 * (BPhat + BPhat')

    up, sp, vp = svd(BPhat)
    inv_sp = Diagonal(sp.^-1);
    inv_sp[inv_sp.>1e+12] .=0;
    
    inv_BPhat = (up * inv_sp * vp')'

    Bnut =vec( AT_draw[end - i + 1, :]')  - PHIF * BAtt' - CONF
    Amean = vec(BAtt) + BPtt * PHIF' * inv_BPhat * Bnut
    Pmean = BPtt - BPtt * PHIF' * inv_BPhat * (BPtt * PHIF')'

    um, sm, vm = svd(Pmean)
    Pmchol = um * sqrt.(Diagonal(sm))
    AT_draw[end - i, :] = (Amean + Pmchol * randn(kn))'
end

# Balanced dataset
At_draw = zeros(nobs,Nq*(p+1));

for kk=1:p+1
    At_draw[nobs,(kk-1)*Nq+1:kk*Nq] = AT_draw[1,kk*Nm+(kk-1)*Nq+1:kk*(Nm+Nq)];
end

for i = 1:nobs-1
    Att  = At_mat[nobs-i,:];
    Ptt  = reshape(Pt_mat[nobs-i,:],Nq*(p+1),Nq*(p+1));

    Phat = Matrix(Symmetric(GAMMAs * Ptt * GAMMAs' + GAMMAu * sig_qq * GAMMAu'));
    
    up, sp, vp = svd(Phat);
    inv_sp = Diagonal(sp.^-1);
    inv_sp[inv_sp.>1e+12] .=0;
        
    inv_Phat = (up*inv_sp*vp')';
    
    nut  = At_draw[nobs-i+1,:] - GAMMAs * Att - GAMMAz * Zm[nobs-i,:]- GAMMAc;
      
    Amean = Att + (Ptt*GAMMAs')*inv_Phat*nut;
    Pmean = Ptt - (Ptt*GAMMAs')*inv_Phat*(Ptt*GAMMAs')';   
  
    # singular value decomposition
    um, sm, vm = svd(Pmean);
    Pmchol = um*Diagonal(sm.^0.5);
    At_draw[nobs-i,:] = (Amean + Pmchol*randn(Nq*(p+1),1))';  
end


BVAR Part

In [ ]:
YY = Matrix{Float64}([[YM At_draw[:,1:Nq]];AT_draw[2:end,1:(Nm+Nq)]])

In [ ]:
nobs_   = size(YY,1)-T0;
spec    = [p T0 1 nv nobs_];

In [ ]:
function gibbs_bvar_mf(Y::Matrix, p::Int, B0,V0,S0, hyperparameters::MN_hyper, n_draws::Int = 2000, burnin::Int = 1000)
    T, k = size(Y)
    T0 = 2*p; # pre-sample size
    Tnobs = T - T0;
    Y_dummy, X_dummy = make_dummy_observations(hyperparameters::MN_hyper,  Y::Matrix{Float64}, p::Int, T0::Int)
    X, Y_target = make_lag_matrix(Y, p)
    # premom = vcat(mean(Y[1:p,:],dims=1),std(Y[1:p,:],dims=1));
    # # Y_dummy, X_dummy = make_dummy_observations(k,p,hyperparameters,premom,1)
    # make_dummy_observations(hyperparameters,  Y, p)
    Td = size(Y_dummy, 1);
    X_f = vcat(X_dummy,X[T0+1-p:end,:]);
    Y_f = vcat(Y_dummy, Y_target[T0+1-p:end,:]);

    V0_inv = inv(V0)
    #B_samples = zeros(burnin+n_draws, k, k*p+1)  # n_draws × k × (k*p+1)
    #Σ_samples = zeros(burnin+n_draws, k, k)  # n_draws × k × k
    B_samples = Array{Float64}(undef, n_draws-burnin, k, k*p+1)  # 3D array
    Σ_samples = Array{Float64}(undef, n_draws-burnin, k, k)  # 3D array

    # 초기값
    B_draw = reshape(vec(X_f\Y_f) , k*p+1,k)'
    B_draw_init = transpose(X_f\Y_f);
    XtX_inv = cholesky(X_f'*X_f) \ I;  # safer than inv()
    U_X = cholesky(Symmetric(XtX_inv)).U     
for s in 1:n_draws
    # Posterior of Σ | B
    E = Y_f - X_f * B_draw';
    Σ_post = Wishart(T+Td-k-1-(T0-p),E'E); # Posterior of Σ | Y;
    Σ_draw = inv(rand(Σ_post));
    # posterior of B | Σ 
    # B_V =  Matrix(Symmetric(kron(Σ_draw,XtX_inv)));
    # B_draw = reshape(vec(X_f\Y_f) + cholesky(B_V).U *randn(prod(size(B0))), k,k*p+1);        
    B_draw = B_draw_init + cholesky(Matrix(Symmetric(Σ_draw))).L *randn((size(B0))) *U_X; # A Secret Sauce for Super-fast computation, by ChatGPT
    # Save results
#    B_samples[s,:,:] = reshape(B_draw',1,k,k*p+1);
    #Σ_samples[s,:,:] = reshape(Σ_draw,1,k,k);
    if s>burnin
    @views B_samples[s-burnin, :, :] .= B_draw;
    @views Σ_samples[s-burnin, :, :] .= Σ_draw;
    end
 if mod(s,5000) == 0
        println("Iteration: ", s)
    end
  
end
    return B_samples, Σ_samples;
end